In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
import cifar10
from cifar10 import num_classes
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [11]:
train_data,train_labels,onehot_train=cifar10.load_training_data()
test_data,test_labels,onehot_test=cifar10.load_test_data()
train_data=train_data.astype('float32')
train_labels=train_labels.astype('int32')
test_data=test_data.astype('float32')
test_labels=test_labels.astype('int32')
tf.logging.set_verbosity(tf.logging.ERROR)
# tf.logging.set_verbosity(tf.logging.INFO)

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [28]:
weight_decay = 1e-4
regularizer = tf.contrib.layers.l2_regularizer(scale=weight_decay)
filter_size = 3
feature_maps = 32
acc_track = []

In [29]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 32, 32, 3])
  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
#   print("******************************")
#   print(pool2.shape)
  pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits/5, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [30]:
def current_loss(flag):
    train_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      shuffle=False)
    eval_result1=cifar_classifier.evaluate(input_fn=train_eval_input_fn)
    if flag:
        print("%%%%%%% Train accuracy %%%%%%%%%%%%\n",eval_result1)
    [a,b,c] = eval_result1.items()

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_result2=cifar_classifier.evaluate(input_fn=eval_input_fn)
    if flag:
        print("######### Test accuracy #############\n",eval_result2)
    [d,e,f] = eval_result2.items()
    acc_track.append([a[1],d[1]])
    return a[1],d[1]

In [31]:
# Evaluate the model and print results
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./cifar_convnet_model_test2")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
print("started\n")
# Train the model
i = 0 
while 1:
    if i:
        curr_result = current_loss(1)
        print(curr_result)
        if curr_result[0] - curr_result[1] > 0.10 or curr_result[1] > 0.82:
            break
# Train the model
    print("Training")
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=256,
      num_epochs=None,
      shuffle=True)

    cifar_classifier.train(
      input_fn=train_input_fn,
      steps=1000,
      hooks=None)
    i=1
    #print(curr_result)
    
print ("Finished")

started

Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.29564, 'loss': 2.00612, 'global_step': 2002}
######### Test accuracy #############
 {'accuracy': 0.3011, 'loss': 2.0048447, 'global_step': 2002}
(0.29564, 0.3011)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.32976, 'loss': 1.9205403, 'global_step': 3002}
######### Test accuracy #############
 {'accuracy': 0.3332, 'loss': 1.918425, 'global_step': 3002}
(0.32976, 0.3332)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.35074, 'loss': 1.8628188, 'global_step': 4002}
######### Test accuracy #############
 {'accuracy': 0.3566, 'loss': 1.8613126, 'global_step': 4002}
(0.35074, 0.3566)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.37396, 'loss': 1.8056538, 'global_step': 5002}
######### Test accuracy #############
 {'accuracy': 0.3786, 'loss': 1.8044589, 'global_step': 5002}
(0.37396, 0.3786)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.39112, 'loss': 1.748781, 'glo

######### Test accuracy #############
 {'accuracy': 0.5701, 'loss': 1.2053485, 'global_step': 38002}
(0.60184, 0.5701)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.60696, 'loss': 1.1366259, 'global_step': 39002}
######### Test accuracy #############
 {'accuracy': 0.5765, 'loss': 1.1927736, 'global_step': 39002}
(0.60696, 0.5765)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.61176, 'loss': 1.1245946, 'global_step': 40002}
######### Test accuracy #############
 {'accuracy': 0.5803, 'loss': 1.1821471, 'global_step': 40002}
(0.61176, 0.5803)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.61776, 'loss': 1.1159326, 'global_step': 41002}
######### Test accuracy #############
 {'accuracy': 0.5855, 'loss': 1.1767179, 'global_step': 41002}
(0.61776, 0.5855)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.6184, 'loss': 1.115605, 'global_step': 42002}
######### Test accuracy #############
 {'accuracy': 0.5844, 'loss': 1.1790789, 'global_ste

######### Test accuracy #############
 {'accuracy': 0.6453, 'loss': 1.0138613, 'global_step': 74002}
(0.71392, 0.6453)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.7102, 'loss': 0.8601084, 'global_step': 75002}
######### Test accuracy #############
 {'accuracy': 0.6447, 'loss': 1.0184146, 'global_step': 75002}
(0.7102, 0.6447)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.7152, 'loss': 0.8443337, 'global_step': 76002}
######### Test accuracy #############
 {'accuracy': 0.6453, 'loss': 1.0094373, 'global_step': 76002}
(0.7152, 0.6453)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.71938, 'loss': 0.8385902, 'global_step': 77002}
######### Test accuracy #############
 {'accuracy': 0.6489, 'loss': 1.0053575, 'global_step': 77002}
(0.71938, 0.6489)
Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.71964, 'loss': 0.8344712, 'global_step': 78002}
######### Test accuracy #############
 {'accuracy': 0.6473, 'loss': 1.0067649, 'global_step'

In [ ]:
current_loss(1)